<a href="https://colab.research.google.com/gist/JetBelson/7b6bb283779dbe358ccf40b2a368b150/transactionfrauddetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<body style="background-color:black;">

<H1 style="color:#00B271" size=80 align="center"> Transaction Fraud Detection with Knowledge Graph </H1>


Created by [Jet ZHENG](https:www.jetzheng.com)

Disclaimer - This project provides information and discussion about the procedures on transaction fraud detection. All contents provided in this project, and in any linked mateirals, are intended for learning purpose only. Any users should comply with all barckgroud IP involved in this project.  

Acknowledgement: This projects was adapted from [Neo4J](https://neo4j.com/).

# Package

In [ ]:
from neo4j import GraphDatabase, basic_auth

import pandas as pd


In [ ]:
URI =  "bolt://localhost:11007"
AUTH = basic_auth("neo4j", "zzj")



driver = GraphDatabase.driver(
    uri=URI,
    auth=AUTH,
    max_connection_lifetime=30 * 60,
    max_connection_pool_size=50,
    connection_acquisition_timeout=2 * 60)


def neo4j_data_print(res):
    print("")
    data = []
    for ri in res:
        data.append(ri.values())
    df = pd.DataFrame(columns=res[0].keys(), data=data)
    print(df)

# Tabale of Contents


* Backgourd  

* Data Exploration  

* First-party Fraud Detection  

* Second-party Fraud (Money Mules)

# Backgroud


## 1.1 Problem Definition  

#### What is Fraud?
Fraud occurs when an individual or group of individuals, or a business entity intentionally deceives another individual or business entity with misrepresentation of identity, products, services, or financial transactions and/or false promises with no intention of fulfilling them.

#### Fraud Categories
* First-party Fraud

An individual, or group of individuals, misrepresent their identity or give false information when applying for a product or services to receive more favourable rates or when have no intention of repayment.

* Second-party Fraud

An individual knowingly gives their identity or personal information to another individual to commit fraud or someone is perpetrating fraud in his behalf.

* Third-party Fraud

An individual, or a group of individuals, create or use another person’s identity, or personal details, to open or takeover an account.  


## 1.2 Aim
This project aims to detect and label two types of fraudsters: (1)first-party fraudster and (2) money mules by using machine learning with knowledge graph. 

# Data Exploration

This project uses a dataset generated from PaySim developed by [Lopez-Rojas, Elmire, and Axelsson](https://www.researchgate.net/publication/313138956_PAYSIM_A_FINANCIAL_MOBILE_MONEY_SIMULATOR_FOR_FRAUD_DETECTION) and modified by [Dave Voutila](https://github.com/voutilad/paysim).  
PaySim is a financial simulator that simulates mobile money transactions, which was developed with agent-based simulation technqiues and mathematical statistics on a anoymous real-life dataset.  

In [ ]:
# Origial data samples generated with PaySim developed by Lopez-Rojas, Elmire, 
# and Axelsson
df =pd.read_csv('data/paysim_sample.csv')
df.head()

,Unnamed: 0,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


#### 2.1 Basic information extracted from origianl PaySim.  

**Agent** type:
* **Clitent:** model the end users in the mobile money network, effectively mapping to unique accounts that, in theory, are controlled by real people. 
* **Merchants:** model the vendors or businesses that participate in the network through interactions with Clients.
* **Banks:** act only as a target for Debit transactions.  

**Transaction** types:
* **CASH-IN** is the process of increasing the balance of account by paying in cash to a merchant.
* **CASH-OUT** means to withdraw cash from a merchant which decreases the balance of the account
* **DEBIT** is similar process than CASH-OUT and involves sending the money from the mobile money service to a bank account
* **PAYMENT** is the process of paying for goods or services to merchants which decreases the balance of the account and increases the balance of the receiver.
* **TRANSFER** is the process of sending money to another user of the service through the mobile money platform.  
  
Graphical representation of the original PaySim data model  
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/simplified-data-model.png?raw=true" width="850"/></div>
  
  
  
#### 2.2 Improving PaySim
* improving ergnomics and usability of PaySim, allowing us to enhance it and add new features
* expanding upon the modeling of Fraudsters, incorporating the two common types of fraudsters: 1st and 3rd party

Graphical representation of the improved PaySim data model  

<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/paysim-2.1.0-part1.png?raw=true" width="850"/></div>


The whole neo4j data schema is visualizd ad below. And the neo4j database used in this project can be downloaded from [Github](https://github.com/neo4j-graph-examples/fraud-detection/tree/main/data).  
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-schema-visualizaiton.png?raw=true" width="850"/></div>


## 


In [ ]:
# Check dataset statistic information
with driver.session() as sesssion:
    res = sesssion.execute_read(lambda tx: tx.run("""
            CALL apoc.meta.stats()
            YIELD labelCount,relTypeCount,propertyKeyCount,nodeCount,relCount
            RETURN labelCount,relTypeCount,propertyKeyCount,nodeCount,relCount
            """
        ).data()
    )
    neo4j_data_print(res)



   labelCount  relTypeCount  propertyKeyCount  nodeCount  relCount
0          13             9                11     332973    980857


In [ ]:
# List dnode labels
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            CALL db.labels() YIELD label
            CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as count', {})
            YIELD value
            RETURN label as Label, value.count AS Count
            """
        ).data()
    )
    neo4j_data_print(res)


# List relationship types
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            CALL db.relationshipTypes() YIELD relationshipType as type
            CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as count', {})
            YIELD value
            RETURN type AS Relationship, value.count AS Count
            """
        ).data()
    )
    print("")
    neo4j_data_print(res)


# List transaction types
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH (t:Transaction)
            WITH count(t) AS globalCnt
            UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
            CALL apoc.cypher.run('MATCH (t:' + txType + ')
                RETURN count(t) AS txCnt', {})
            YIELD value
            RETURN txType, value.txCnt AS NumberOfTransactions,
            round(toFloat(value.txCnt)/toFloat(globalCnt)*100, 2) AS `%Transactions`
            ORDER BY `%Transactions` DESC;
            """
        ).data()
    )
    print("")
    neo4j_data_print(res)





          Label   Count
0        Client    2433
1          Bank       3
2      Merchant     347
3          Mule     433
4        CashIn  149037
5       CashOut   76023
6         Debit    4392
7       Payment   74577
8      Transfer   19460
9   Transaction  323489
10        Email    2229
11          SSN    2238
12        Phone    2234


         Relationship   Count
0           PERFORMED  323489
1                  TO  323489
2             HAS_SSN    2433
3           HAS_EMAIL    2433
4           HAS_PHONE    2433
5            FIRST_TX    2332
6             LAST_TX    2332
7                NEXT  321157
8  SHARED_IDENTIFIERS     759


     txType  NumberOfTransactions  %Transactions
0    CashIn                149037          46.07
1   CashOut                 76023          23.50
2   Payment                 74577          23.05
3  Transfer                 19460           6.02
4     Debit                  4392           1.36


# First-party Fraud Detection

Synthetic identity fraud and first party fraud can be identified by performing entity link analysis to detect identities linked to other identities via shared PII.

There are three types of **personally identifiable information (PII)** in this dataset - SSN, Email and Phone Number


<font color=#00B271 > Our hypothesis is that clients who share identifiers are suspicious and have a higher potential to commit fraud. However, all shared identifier links are not suspicious, for example, two people sharing an email address. Hence, we compute a fraud score based on shared PII relationships and label the top X percentile clients as fraudsters.</font>

We will first identify clients that share identifiers and create a new relationship between clients that share identifiers

In [ ]:
# Identify clients sharing PII
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(n) <-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
            WHERE id(c1) < id(c2)
            RETURN c1.id, c2.id, count(*) AS freq
            ORDER BY freq DESC;
            """
        ).data()
    )
    print('clients:')
    neo4j_data_print(res)


# Number of unique clients sharing PII
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(n) <-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
            WHERE id(c1) <> id(c2)
            RETURN count(DISTINCT c1.id) AS freq;
            """
        ).data()
    )
    print('unique clients:')
    neo4j_data_print(res)


clients:

                c1.id             c2.id  freq
0    4952527271473904  4816336012071985     3
1    4883445100935916  4708373581412325     3
2    4658150168863397  4100374538108184     3
3    4673951123644611  4795773320377768     3
4    4192214340630620  4912097363222923     3
..                ...               ...   ...
754  4910140986334626  4114683318919154     1
755  4454780847105236  4210575070378533     1
756  4721862020593706  4210575070378533     1
757  4445521165797820  4210575070378533     1
758  4884394011157286  4210575070378533     1

[759 rows x 3 columns]
unique clients:

   freq
0   336


In [ ]:
# Create a new relationship to connect clients that share identifiers and add 
# the number of shared identifiers as a property on that relationship
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH (c1:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN] ->(n)<- [:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]-(c2:Client)
            WHERE id(c1) < id(c2)
            WITH c1, c2, count(*) as cnt
            MERGE (c1) - [:SHARED_IDENTIFIERS {count: cnt}] -> (c2);
            """
        ).data()
    )

with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH p = (:Client) - [s:SHARED_IDENTIFIERS] -> (:Client) WHERE s.count >= 2 RETURN p limit 25;
            """
        ).data()
    )

<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-share-pii.png?raw=true" width="1500"/></div>


We will construct a workflow with graph algorithms to detect fraud rings, score clients based on the number of common connections and rank them to select the top few suspicious clients and label them as fraudsters.

* Identify clusters of clients sharing PII using a community detection algorithm (Weakly Connected Components)

* Find similar clients within the clusters using pairwise similarity algorithms (Node Similarity)

* Calculate and assign fraud score to clients using centrality algorithms (Degree Centrality) and

* Use computed fraud scores to label clients as potential fraudsters

In [ ]:
# Project graph to memory for computing
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            CALL gds.graph.project(
                'wcc',
                {
                    Client: {
                        label: 'Client'
                    }
                },
                {
                    SHARED_IDENTIFIERS:{
                        type: 'SHARED_IDENTIFIERS',
                        orientation: 'UNDIRECTED',
                        properties: {
                            count: {
                                property: 'count'
                            }
                        }
                    }
                }
            ) YIELD graphName,nodeCount,relationshipCount,projectMillis;
            """
        )
    )


#### 3.1 Cluster identification with weekly connected component

**Weakly Connected Components (WCC)** is used to find groups of connected nodes, where all nodes in the same set form a connected component. WCC is often used early in an analysis understand the structure of a graph.

In [ ]:
# Run weakly connected components to find clusters of clients sharing PII.
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            CALL gds.wcc.stream(
                'wcc',
                {
                    nodeLabels: ['Client'],
                    relationshipTypes: ['SHARED_IDENTIFIERS'],
                    consecutiveIds: true
                }
            )
            YIELD nodeId, componentId
            RETURN gds.util.asNode(nodeId).id AS clientId, componentId
            ORDER BY componentId LIMIT 20
            """
        ).data()
    )

    neo4j_data_print(res)


            clientId  componentId
0   4997933060327094            0
1   4776276949898423            1
2   4858607188760216            2
3   4287186486553145            3
4   4661202154682409            4
5   4649268238636650            5
6   4426707672690219            6
7   4922246870240518            7
8   4415848797892554            8
9   4548497513788330            9
10  4950634022082174           10
11  4860903977910377           11
12  4234798486577769           12
13  4175792657809755           13
14  4361287590543243           14
15  4833833649287561           15
16  4495151043368906           16
17  4028666746330768           17
18  4664274185403862           18
19  4699974121949998           19


In [ ]:
# write results to the database
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.wcc.stream('wcc',
                {
                    nodeLabels: ['Client'],
                    relationshipTypes: ['SHARED_IDENTIFIERS'],
                    consecutiveIds: true
                }
            )
            YIELD componentId, nodeId
            WITH componentId AS cluster, gds.util.asNode(nodeId) AS client
            WITH cluster, collect(client.id) AS clients
            WITH cluster, clients, size(clients) AS clusterSize WHERE clusterSize > 1
            UNWIND clients AS client
            MATCH (c:Client) WHERE c.id = client
            SET c.firstPartyFraudGroup=cluster;
            """
        ).data()
    )




In [ ]:
# collect cluster results and visualize
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH (c:Client)
            WITH c.firstPartyFraudGroup AS fpGroupID, collect(c.id) AS fGroup
            WITH *, size(fGroup) AS groupSize WHERE groupSize >= 9
            WITH collect(fpGroupID) AS fraudRings
            MATCH p=(c:Client)-[:HAS_SSN|HAS_EMAIL|HAS_PHONE]->()
            WHERE c.firstPartyFraudGroup IN fraudRings
            RETURN p
            """
        ).data()
    )
    neo4j_data_print(res)




                                                     p
0    [{'firstPartyFraudScore': 1.4, 'firstPartyFrau...
1    [{'firstPartyFraudScore': 1.4, 'firstPartyFrau...
2    [{'firstPartyFraudScore': 1.4, 'firstPartyFrau...
3    [{'firstPartyFraudScore': 1.6, 'firstPartyFrau...
4    [{'firstPartyFraudScore': 1.6, 'firstPartyFrau...
..                                                 ...
292  [{'firstPartyFraudScore': 2.1999999999999997, ...
293  [{'firstPartyFraudScore': 2.1999999999999997, ...
294  [{'firstPartyFraudScore': 2.6000000000000005, ...
295  [{'firstPartyFraudScore': 2.6000000000000005, ...
296  [{'firstPartyFraudScore': 2.6000000000000005, ...

[297 rows x 1 columns]


<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-wcc-visualizaiton.png?raw=true" width="1500"/></div>
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-wcc-visualizaiton-1.png?raw=true" width="1500"/></div>


#### 3.2 Pairwise similarity scores for additional context

We have observed that some identifiers (Email/SSN/Phone Number) are connected to more than one client pointing to reuse of identifiers among clients.

We hypothesize that identities that are connected to highly reused identifiers have higher potential to commit fraud.

We could compute pairwise similarity scores using Jaccard metric and build additional relationships to connect clients based on shared identifiers and score these pairs based on Jaccard score.

In [ ]:
# project data to commeory using cypher
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            MATCH (c:Client) WHERE c.firstPartyFraudGroup is not NULL
            WITH collect(c) as clients
            MATCH (n) WHERE n:Email OR n:Phone OR n:SSN
            WITH clients, collect(n) as identifiers
            WITH clients + identifiers as nodes

            MATCH (c:Client)-[:HAS_EMAIL|:HAS_PHONE|:HAS_SSN]->(id)
            WHERE c.firstPartyFraudGroup is not NULL
            WITH nodes, collect({source: c, target: id}) as relationships

            CALL gds.graph.project.cypher('similarity',
                "UNWIND $nodes as n RETURN id(n) AS id,labels(n) AS labels",
                "UNWIND $relationships as r RETURN id(r['source']) AS source, id(r['target']) AS target, 'HAS_IDENTIFIER' as type",
                { parameters: {nodes: nodes, relationships: relationships}}
            )
            YIELD graphName, nodeCount, relationshipCount, projectMillis
            RETURN graphName, nodeCount, relationshipCount, projectMillis
            """
        ).data()
    )
    neo4j_data_print(res)




    graphName  nodeCount  relationshipCount  projectMillis
0  similarity       7037               1008            164


In [ ]:
# Write similarity scores to in-memory graph (Mutate)
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.nodeSimilarity.mutate('similarity',
                {
                    topK:15,
                    mutateProperty: 'jaccardScore',
                    mutateRelationshipType:'SIMILAR_TO'
                }
            )
            """
        ).data()
    )
# Write results from in-memory graph to the Database
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.graph.writeRelationship('similarity', 'SIMILAR_TO', 'jaccardScore');
            """
        )
    )




#### 3.3 Calculate First-party Fraud Score

We compute first party fraud score using weighted degree centrality algorithm.

In this step, we compute and assign fraud score (firstPartyFraudScore) to clients in the clusters identified in previous steps based on SIMILAR_TO relationships weighted by jaccardScore

Weighted degree centrality algorithm add up similarity scores (jaccardScore) on the incoming SIMILAR_TO relationships for a given node in a cluster and assign the sum as the corresponding firstPartyFraudScore. This score represents clients who are similar to many others in the cluster in terms of sharing identifiers. Higher firstPartyFraudScore represents greater potential for committing fraud.

In [ ]:
# Write back centrality scores as firstPartyFraudScore to the database using write mode.

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.degree.write('similarity',
                {
                    nodeLabels: ['Client'],
                    relationshipTypes: ['SIMILAR_TO'],
                    relationshipWeightProperty: 'jaccardScore',
                    writeProperty: 'firstPartyFraudScore'
                }
            )
            """
        )
    )



#### 3.4 attach fraudster labels

In [ ]:
# We find clients with first-party fraud score greater than some threshold (X) 
# and label those top X percentile clients as fraudsters. In this example, 
# using 95th percentile as a threshold, we set a property FirstPartyFraudster 
# on the Client node.


with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH (c:Client)
            WHERE c.firstPartyFraudScore IS NOT NULL
            WITH percentileCont(c.firstPartyFraudScore, 0.95) AS firstPartyFraudThreshold

            MATCH (c:Client)
            WHERE c.firstPartyFraudScore > firstPartyFraudThreshold
            SET c:FirstPartyFraudster
            """
        )
    )


**visualize the FirstPartyFaudsters and their pii**
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-pii.png?raw=true" width="1500"/></div>

# Second-party Fraud (Money Mules)

Criminals recruit money mules to help launder proceeds derived from online scams and frauds. Money mules add layers of distance between victims and fraudsters, which makes it harder for law enforcement to accurately trace money trails.

In this exercise, we detect money mules in the paysim dataset. 

Our hypothesis is that clients who transfer money to/from first party fraudsters are suspects for second party fraud.

#### 4.1 Transactions between first-party fraudsters and client

In [ ]:
# The first step is to find out clients who weren’t identified as first party 
# fraudsters but they transact with first party fraudsters

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH p=(:Client:FirstPartyFraudster)-[]-(:Transaction)-[]-(c:Client)
            WHERE NOT c:FirstPartyFraudster
            RETURN p
            """
        )
    )


**visualize the FirstPartyFaudsters and their transaction networks**
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-transaction-networks.png?raw=true width="1500"/></div>
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-transaction-networks-1.png?raw=true width="1500"/></div>

In [ ]:
# Also, lets find out what types of transactions do these Clients perform with first party fraudsters

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH (:Client:FirstPartyFraudster)-[]-(txn:Transaction)-[]-(c:Client)
            WHERE NOT c:FirstPartyFraudster
            UNWIND labels(txn) AS transactionType
            RETURN transactionType, count(*) AS freq;
            """
        ).data()
    )
    neo4j_data_print(res)



  transactionType  freq
0        Transfer    89
1     Transaction    89


#### 4.2 Create new relationships


Let’s go ahead and create TRANSFER_TO relationships between clients with firstPartyFraudster tags and other clients. Also add the total amount from all such transactions as a property on TRANSFER_TO relationships.

Since the total amount transferred from a fraudster to a client and the total amount transferred in the reverse direction are not the same, we have to create relationships in two separate queries.




In [ ]:
# Create TRANSFER_TO relationship from a fraudster to a client
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH (c1:FirstPartyFraudster)-[]->(t:Transaction)-[]->(c2:Client)
            WHERE NOT c2:FirstPartyFraudster
            WITH c1, c2, sum(t.amount) AS totalAmount
            SET c2:SecondPartyFraudSuspect
            CREATE (c1)-[:TRANSFER_TO {amount:totalAmount}]->(c2);
            """
        )
    )


# Create TRANSFER_TO relationship from a client to a client from a fraudster
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH (c1:FirstPartyFraudster)<-[]-(t:Transaction)<-[]-(c2:Client)
            WHERE NOT c2:FirstPartyFraudster
            WITH c1, c2, sum(t.amount) AS totalAmount
            SET c2:SecondPartyFraudSuspect
            CREATE (c1)<-[:TRANSFER_TO {amount:totalAmount}]-(c2);
            """
        )
    )


In [ ]:
# visiualize the new relationships
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
        MATCH p=(:Client:FirstPartyFraudster)-[:TRANSFER_TO]-(c:Client)
        WHERE NOT c:FirstPartyFraudster
        RETURN p;
            """
        )
    )


<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-SecondPartySuspect.png?raw=true width="1500"/></div>

#### 4.3 Second-party Fraud


Our objective is to find out clients who may have supported the first party fraudsters and were not identified as potential first party fraudsters.

Our hypothesis is that clients who perform transactions of type Transfer where they either send or receive money from first party fraudsters are flagged as suspects for second party fraud.

To identify such clients, make use of TRANSFER_TO relationships and use this recipe:

* Use WCC (community detection) to identify networks of clients who are connected to first party fraudsters

* Use PageRank (centrality) to score clients based on their influence in terms of the amount of money transferred to/from fraudsters

* Assign risk score (secondPartyFraudScore) to these clients

In [ ]:
# Let’s use native projection and create an in-memory graph with Client nodes 
# and TRANSFER_TO relationships.
with driver.session() as sesssion:
    res = sesssion.execute_read(
        lambda tx: tx.run(
            """
            CALL gds.graph.project('SecondPartyFraudNetwork',
                'Client',
                'TRANSFER_TO',
                {relationshipProperties:'amount'}
            );
            """
        )
    )

In [ ]:
# We will see if there are any clusters with more than one clients in them and 
# if there are, then we should add a tag secondPartyFraudGroup to find them 
# later using local queries.

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.wcc.stream('SecondPartyFraudNetwork')
            YIELD nodeId, componentId
            WITH gds.util.asNode(nodeId) AS client, componentId AS clusterId
            WITH clusterId, collect(client.id) AS cluster
            WITH clusterId, size(cluster) AS clusterSize, cluster
            WHERE clusterSize > 1
            UNWIND cluster AS client
            MATCH(c:Client {id:client})
            SET c.secondPartyFraudGroup=clusterId;
            """
        )
    )


In [ ]:
# Use pagerank to find out who among the suspects have relatively higher fraud
# scores. Please note that relationships are weighted by the total amount 
# transferred to fraudsters.

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            CALL gds.pageRank.stream('SecondPartyFraudNetwork',
                {relationshipWeightProperty:'amount'}
            )YIELD nodeId, score
            WITH gds.util.asNode(nodeId) AS client, score AS pageRankScore

            WHERE client.secondPartyFraudGroup IS NOT NULL
                    AND pageRankScore > 0 AND NOT client:FirstPartyFraudster

            MATCH(c:Client {id:client.id})
            SET c:SecondPartyFraud
            SET c.secondPartyFraudScore = pageRankScore;
            """
        )
    )






In [ ]:
# Visualize second party fraud networks

with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
            MATCH p=(:Client:FirstPartyFraudster)-[:TRANSFER_TO]-(c:Client)
            WHERE NOT c:FirstPartyFraudster
            RETURN p;
            """
        )
    )

<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-SecondPartyFaud-Suspect.png?raw=true width="1500"/></div>
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-firstfraudster-SecondPartyFaud-Suspect-1.png?raw=true width="1500"/></div>


  
  
**New Schema After Processing**
<div><img src="https://github.com/JetBelson/images/blob/main/transaction_fraud_detection_imgs/neo4j-schema-visualizaiton-2.png?raw=true width="1500"/></div>


In [ ]:
# clean up graphs in memory
with driver.session() as sesssion:
    res = sesssion.execute_write(
        lambda tx: tx.run(
            """
    CALL gds.graph.list()
    YIELD graphName AS namedGraph
    WITH namedGraph
    CALL gds.graph.drop(namedGraph)
    YIELD graphName
    RETURN graphName;
            """
        )
    )
